# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
import gradio as gr
from anthropic import Anthropic


In [80]:
# Initialization

load_dotenv(override=True)

MODEL = "claude-3-5-sonnet-latest"
claude_key = os.getenv('CLAUDE_API_KEY')
anthropic_client = Anthropic(api_key=claude_key)


In [81]:
system_message = "You are a concise, helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. Respond only with the final answer, no explanations, and do not show your reasoning steps or thinking. "
system_message += "Always be accurate. If you don't know the answer, say so."
system_message += "You have access to tools, but only use them when necessary. If a tool is not required, respond as normal"

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [82]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [86]:
get_ticket_price("London")  # Example call to test the function

Tool get_ticket_price called for London


'$799'

In [83]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "input_schema": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [ ]:
airline_schedule_schema = {
    "type": "object",
    "properties": {
        "source_city": {
            "type": "string",
            "enum": ["London", "Tokyo", "Berlin", "Paris"],
            "description": "City where the flight originates"
        },
        "destination_city": {
            "type": "string",
            "enum": ["Miami", "Sacramento", "San Jose", "Portland"],
            "description": "City where the flight lands"
        },
        "schedules": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "day": {
                        "type": "string",
                        "enum": ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
                    },
                    "flights": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "flight_number": {"type": "string"},
                                "departure_time": {"type": "string", "pattern": "^([01]\\d|2[0-3]):[0-5]\\d$"},
                                "arrival_time": {"type": "string", "pattern": "^([01]\\d|2[0-3]):[0-5]\\d$"}
                            },
                            "required": ["flight_number", "departure_time", "arrival_time"]
                        },
                        "minItems": 4,
                        "maxItems": 4
                    }
                },
                "required": ["day", "flights"]
            },
            "minItems": 7,
            "maxItems": 7
        }
    },
    "required": ["source_city", "destination_city", "schedules"]
}
airline_schedule_london_miami = {
    "source_city": "London",
    "destination_city": "Miami",
    "schedules": [
        {
            "day": "Monday",
            "flights": [
                {"flight_number": "FA101", "departure_time": "08:00", "arrival_time": "14:00"},
                {"flight_number": "FA102", "departure_time": "12:00", "arrival_time": "18:00"},
                {"flight_number": "FA103", "departure_time": "16:00", "arrival_time": "22:00"},
                {"flight_number": "FA104", "departure_time": "20:00", "arrival_time": "02:00"}
            ]
        },
        {
            "day": "Tuesday",
            "flights": [
                {"flight_number": "FA201", "departure_time": "08:00", "arrival_time": "14:00"},
                {"flight_number": "FA202", "departure_time": "12:00", "arrival_time": "18:00"},
                {"flight_number": "FA203", "departure_time": "16:00", "arrival_time": "22:00"},
                {"flight_number": "FA204", "departure_time": "20:00", "arrival_time": "02:00"}
            ]
        },
        {
            "day": "Wednesday",
            "flights": [
                {"flight_number": "FA301", "departure_time": "08:00", "arrival_time": "14:00"},
                {"flight_number": "FA302", "departure_time": "12:00", "arrival_time": "18:00"},
                {"flight_number": "FA303", "departure_time": "16:00", "arrival_time": "22:00"},
                {"flight_number": "FA304", "departure_time": "20:00", "arrival_time": "02:00"}
            ]
        },
        {
            "day": "Thursday",
            "flights": [
                {"flight_number": "FA401", "departure_time": "08:00", "arrival_time": "14:00"},
                {"flight_number": "FA402", "departure_time": "12:00", "arrival_time": "18:00"},
                {"flight_number": "FA403", "departure_time": "16:00", "arrival_time": "22:00"},
                {"flight_number": "FA404", "departure_time": "20:00", "arrival_time": "02:00"}
            ]
        },
        {
            "day": "Friday",
            "flights": [
                {"flight_number": "FA501", "departure_time": "08:00", "arrival_time": "14:00"},
                {"flight_number": "FA502", "departure_time": "12:00", "arrival_time": "18:00"},
                {"flight_number": "FA503", "departure_time": "16:00", "arrival_time": "22:00"},
                {"flight_number": "FA504", "departure_time": "20:00", "arrival_time": "02:00"}
            ]
        },
        {
            "day": "Saturday",
            "flights": [
                {"flight_number": "FA601", "departure_time": "08:00", "arrival_time": "14:00"},
                {"flight_number": "FA602", "departure_time": "12:00", "arrival_time": "18:00"},
                {"flight_number": "FA603", "departure_time": "16:00", "arrival_time": "22:00"},
                {"flight_number": "FA604", "departure_time": "20:00", "arrival_time": "02:00"}
            ]
        },
        {
            "day": "Sunday",
            "flights": [
                {"flight_number": "FA701", "departure_time": "08:00", "arrival_time": "14:00"},
                {"flight_number": "FA702", "departure_time": "12:00", "arrival_time": "18:00"},
                {"flight_number": "FA703", "departure_time": "16:00", "arrival_time": "22:00"},
                {"flight_number": "FA704", "departure_time": "20:00", "arrival_time": "02:00"}
            ]
        }
    ]
}

airline_schedule_paris_sacramento = {
    "source_city": "Paris",
    "destination_city": "Sacramento",
    "schedules": [
        {
            "day": "Monday",
            "flights": [
                {"flight_number": "FB101", "departure_time": "09:00", "arrival_time": "15:00"},
                {"flight_number": "FB102", "departure_time": "13:00", "arrival_time": "19:00"},
                {"flight_number": "FB103", "departure_time": "17:00", "arrival_time": "23:00"},
                {"flight_number": "FB104", "departure_time": "21:00", "arrival_time": "03:00"}
            ]
        },
        {
            "day": "Tuesday",
            "flights": [
                {"flight_number": "FB201", "departure_time": "09:00", "arrival_time": "15:00"},
                {"flight_number": "FB202", "departure_time": "13:00", "arrival_time": "19:00"},
                {"flight_number": "FB203", "departure_time": "17:00", "arrival_time": "23:00"},
                {"flight_number": "FB204", "departure_time": "21:00", "arrival_time": "03:00"}
            ]
        },
        {
            "day": "Wednesday",
            "flights": [
                {"flight_number": "FB301", "departure_time": "09:00", "arrival_time": "15:00"},
                {"flight_number": "FB302", "departure_time": "13:00", "arrival_time": "19:00"},
                {"flight_number": "FB303", "departure_time": "17:00", "arrival_time": "23:00"},
                {"flight_number": "FB304", "departure_time": "21:00", "arrival_time": "03:00"}
            ]
        },
        {
            "day": "Thursday",
            "flights": [
                {"flight_number": "FB401", "departure_time": "09:00", "arrival_time": "15:00"},
                {"flight_number": "FB402", "departure_time": "13:00", "arrival_time": "19:00"},
                {"flight_number": "FB403", "departure_time": "17:00", "arrival_time": "23:00"},
                {"flight_number": "FB404", "departure_time": "21:00", "arrival_time": "03:00"}
            ]
        },
        {
            "day": "Friday",
            "flights": [
                {"flight_number": "FB501", "departure_time": "09:00", "arrival_time": "15:00"},
                {"flight_number": "FB502", "departure_time": "13:00", "arrival_time": "19:00"},
                {"flight_number": "FB503", "departure_time": "17:00", "arrival_time": "23:00"},
                {"flight_number": "FB504", "departure_time": "21:00", "arrival_time": "03:00"}
            ]
        },
        {
            "day": "Saturday",
            "flights": [
                {"flight_number": "FB601", "departure_time": "09:00", "arrival_time": "15:00"},
                {"flight_number": "FB602", "departure_time": "13:00", "arrival_time": "19:00"},
                {"flight_number": "FB603", "departure_time": "17:00", "arrival_time": "23:00"},
                {"flight_number": "FB604", "departure_time": "21:00", "arrival_time": "03:00"}
            ]
        },
        {
            "day": "Sunday",
            "flights": [
                {"flight_number": "FB701", "departure_time": "09:00", "arrival_time": "15:00"},
                {"flight_number": "FB702", "departure_time": "13:00", "arrival_time": "19:00"},
                {"flight_number": "FB703", "departure_time": "17:00", "arrival_time": "23:00"},
                {"flight_number": "FB704", "departure_time": "21:00", "arrival_time": "03:00"}
            ]
        }
    ]
}

airline_schedule_tokyo_sanjose = {
    "source_city": "Tokyo",
    "destination_city": "San Jose",
    "schedules": [
        {
            "day": "Monday",
            "flights": [
                {"flight_number": "FC101", "departure_time": "07:00", "arrival_time": "13:00"},
                {"flight_number": "FC102", "departure_time": "11:00", "arrival_time": "17:00"},
                {"flight_number": "FC103", "departure_time": "15:00", "arrival_time": "21:00"},
                {"flight_number": "FC104", "departure_time": "19:00", "arrival_time": "01:00"}
            ]
        },
        {
            "day": "Tuesday",
            "flights": [
                {"flight_number": "FC201", "departure_time": "07:00", "arrival_time": "13:00"},
                {"flight_number": "FC202", "departure_time": "11:00", "arrival_time": "17:00"},
                {"flight_number": "FC203", "departure_time": "15:00", "arrival_time": "21:00"},
                {"flight_number": "FC204", "departure_time": "19:00", "arrival_time": "01:00"}
            ]
        },
        {
            "day": "Wednesday",
            "flights": [
                {"flight_number": "FC301", "departure_time": "07:00", "arrival_time": "13:00"},
                {"flight_number": "FC302", "departure_time": "11:00", "arrival_time": "17:00"},
                {"flight_number": "FC303", "departure_time": "15:00", "arrival_time": "21:00"},
                {"flight_number": "FC304", "departure_time": "19:00", "arrival_time": "01:00"}
            ]
        },
        {
            "day": "Thursday",
            "flights": [
                {"flight_number": "FC401", "departure_time": "07:00", "arrival_time": "13:00"},
                {"flight_number": "FC402", "departure_time": "11:00", "arrival_time": "17:00"},
                {"flight_number": "FC403", "departure_time": "15:00", "arrival_time": "21:00"},
                {"flight_number": "FC404", "departure_time": "19:00", "arrival_time": "01:00"}
            ]
        },
        {
            "day": "Friday",
            "flights": [
                {"flight_number": "FC501", "departure_time": "07:00", "arrival_time": "13:00"},
                {"flight_number": "FC502", "departure_time": "11:00", "arrival_time": "17:00"},
                {"flight_number": "FC503", "departure_time": "15:00", "arrival_time": "21:00"},
                {"flight_number": "FC504", "departure_time": "19:00", "arrival_time": "01:00"}
            ]
        },
        {
            "day": "Saturday",
            "flights": [
                {"flight_number": "FC601", "departure_time": "07:00", "arrival_time": "13:00"},
                {"flight_number": "FC602", "departure_time": "11:00", "arrival_time": "17:00"},
                {"flight_number": "FC603", "departure_time": "15:00", "arrival_time": "21:00"},
                {"flight_number": "FC604", "departure_time": "19:00", "arrival_time": "01:00"}
            ]
        },
        {
            "day": "Sunday",
            "flights": [
                {"flight_number": "FC701", "departure_time": "07:00", "arrival_time": "13:00"},
                {"flight_number": "FC702", "departure_time": "11:00", "arrival_time": "17:00"},
                {"flight_number": "FC703", "departure_time": "15:00", "arrival_time": "21:00"},
                {"flight_number": "FC704", "departure_time": "19:00", "arrival_time": "01:00"}
            ]
        }
    ]
}

airline_schedule_berlin_portland = {
    "source_city": "Berlin",
    "destination_city": "Portland",
    "schedules": [
        {
            "day": "Monday",
            "flights": [
                {"flight_number": "FD101", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FD102", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FD103", "departure_time": "18:00", "arrival_time": "00:00"},
                {"flight_number": "FD104", "departure_time": "22:00", "arrival_time": "04:00"}
            ]
        },
        {
            "day": "Tuesday",
            "flights": [
                {"flight_number": "FD201", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FD202", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FD203", "departure_time": "18:00", "arrival_time": "00:00"},
                {"flight_number": "FD204", "departure_time": "22:00", "arrival_time": "04:00"}
            ]
        },
        {
            "day": "Wednesday",
            "flights": [
                {"flight_number": "FD301", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FD302", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FD303", "departure_time": "18:00", "arrival_time": "00:00"},
                {"flight_number": "FD304", "departure_time": "22:00", "arrival_time": "04:00"}
            ]
        },
        {
            "day": "Thursday",
            "flights": [
                {"flight_number": "FD401", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FD402", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FD403", "departure_time": "18:00", "arrival_time": "00:00"},
                {"flight_number": "FD404", "departure_time": "22:00", "arrival_time": "04:00"}
            ]
        },
        {
            "day": "Friday",
            "flights": [
                {"flight_number": "FD501", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FD502", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FD503", "departure_time": "18:00", "arrival_time": "00:00"},
                {"flight_number": "FD504", "departure_time": "22:00", "arrival_time": "04:00"}
            ]
        },
        {
            "day": "Saturday",
            "flights": [
                {"flight_number": "FD601", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FD602", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FD603", "departure_time": "18:00", "arrival_time": "00:00"},
                {"flight_number": "FD604", "departure_time": "22:00", "arrival_time": "04:00"}
            ]
        },
        {
            "day": "Sunday",
            "flights": [
                {"flight_number": "FD701", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FD702", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FD703", "departure_time": "18:00", "arrival_time": "00:00"},
                {"flight_number": "FD704", "departure_time": "22:00", "arrival_time": "04:00"}
            ]
        }
    ]
}

airline_schedule_london_sanjose = {
    "source_city": "London",
    "destination_city": "San Jose",
    "schedules": [
        {
            "day": "Monday",
            "flights": [
                {"flight_number": "FE101", "departure_time": "06:00", "arrival_time": "12:00"},
                {"flight_number": "FE102", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FE103", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FE104", "departure_time": "18:00", "arrival_time": "00:00"}
            ]
        },
        {
            "day": "Tuesday",
            "flights": [
                {"flight_number": "FE201", "departure_time": "06:00", "arrival_time": "12:00"},
                {"flight_number": "FE202", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FE203", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FE204", "departure_time": "18:00", "arrival_time": "00:00"}
            ]
        },
        {
            "day": "Wednesday",
            "flights": [
                {"flight_number": "FE301", "departure_time": "06:00", "arrival_time": "12:00"},
                {"flight_number": "FE302", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FE303", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FE304", "departure_time": "18:00", "arrival_time": "00:00"}
            ]
        },
        {
            "day": "Thursday",
            "flights": [
                {"flight_number": "FE401", "departure_time": "06:00", "arrival_time": "12:00"},
                {"flight_number": "FE402", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FE403", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FE404", "departure_time": "18:00", "arrival_time": "00:00"}
            ]
        },
        {
            "day": "Friday",
            "flights": [
                {"flight_number": "FE501", "departure_time": "06:00", "arrival_time": "12:00"},
                {"flight_number": "FE502", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FE503", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FE504", "departure_time": "18:00", "arrival_time": "00:00"}
            ]
        },
        {
            "day": "Saturday",
            "flights": [
                {"flight_number": "FE601", "departure_time": "06:00", "arrival_time": "12:00"},
                {"flight_number": "FE602", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FE603", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FE604", "departure_time": "18:00", "arrival_time": "00:00"}
            ]
        },
        {
            "day": "Sunday",
            "flights": [
                {"flight_number": "FE701", "departure_time": "06:00", "arrival_time": "12:00"},
                {"flight_number": "FE702", "departure_time": "10:00", "arrival_time": "16:00"},
                {"flight_number": "FE703", "departure_time": "14:00", "arrival_time": "20:00"},
                {"flight_number": "FE704", "departure_time": "18:00", "arrival_time": "00:00"}
            ]
        }
    ]
}

def get_schedules(source_city):
    schedules = []
    for schedule in [
        airline_schedule_london_miami,
        airline_schedule_paris_sacramento,
        airline_schedule_tokyo_sanjose,
        airline_schedule_berlin_portland,
        airline_schedule_london_sanjose
    ]:
        if schedule["source_city"].lower() == source_city.lower():
            schedules.append(schedule)
    return schedules
    

In [85]:
# There's a particular dictionary structure that's required to describe our function:

show_schedules_function = {
    "name": "get_schedules",
    "description": "Get the flight schedules from a given source city to all possible destination cities. Call this whenever you need to know the flight schedules, for example when a customer asks 'What flights are available from this city?'",
    "input_schema": {
        "type": "object",
        "properties": {
            "source_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["source_city"],
        "additionalProperties": False
    }
}

In [86]:
# And this is included in a list of tools:

tools = [price_function, show_schedules_function]

## Getting Anthropic to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [87]:

def chat(message, history):
    
    history = [{k: v for k, v in d.items() if k not in ("metadata", "options")} for d in history]
    messages = history + [{"role": "user", "content": message}]
    response = anthropic_client.messages.create(model=MODEL, max_tokens = 1000,system=system_message, messages=messages, tools=tools)
    messages.append(
        {"role": "assistant", "content": response.content}
    )  # Append the original message to maintain context
    # Check if the response indicates a tool use
     # If the response indicates a tool use, handle it  
    
    if response.stop_reason == "tool_use":
        tool_use = response.content[-1]
        tool_name = tool_use.name
        tool_input = tool_use.input
       # message = response.content[0].text
        result = handle_tool_call(tool_name, tool_input)
       
        # Append the tool result to the messages
        messages.append(
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id":tool_use.id,
                        "content": str(result),
                    },
                ],
            }
        )
        response = anthropic_client.messages.create(model=MODEL, max_tokens = 1000,system=system_message, messages=messages)
    return response.content[0].text
    

In [88]:
# We have to write that function handle_tool_call:

def handle_tool_call(name, input):
    if (name == "get_ticket_price"):
        city = input["destination_city"]
        price = get_ticket_price(city)
        return json.dumps({"destination_city": city,"price": price})
    elif (name == "get_schedules"):
        city = input["source_city"]
        schedule = get_schedules(city)
        return json.dumps({"source_city": city, "schedule": schedule})
    else: 
        price = "Unknown"
    return json.dumps({"destination_city": city,"price": price})

In [92]:
gr.ChatInterface(fn=chat, type="messages").launch(share=True)

* Running on local URL:  http://127.0.0.1:7924
* Running on public URL: https://ee0ba4241a9310d584.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Berlin
Tool get_ticket_price called for Paris
